In [2]:
import requests
import pprint
import json
import urllib
import pandas as pd
import pprint
import time

link = 'https://www.zillow.com/search/GetSearchPageState.htm?'
params = {
    "wants": {"cat1": ["listResults"], "cat2": ["total"]},
    "requestId": 5,

    "searchQueryState": {"pagination": {"currentPage": 1}, "usersSearchTerm": "Dallas TX", "mapBounds": {"west": -97.43455125097657, "east": -96.12031174902344, "south": 32.431040510618146, "north": 33.20313651601504}, "regionSelection": [{"regionId": 38128, "regionType": 6}], "isMapVisible": "false", "filterState": {"sortSelection": {"value": "globalrelevanceex"}}, "isListVisible": "true"}

}

#util
def house_formatter(house):

    keyValues = [
        "addressStreet",
        "availabilityDate",
        "badgeInfo",
        'has3DModel',
        'hasAdditionalAttributions',
        "hasImage",
        'hasOpenHouse',
        'hasVideo',
        "unformattedPrice",
        'isFeaturedListing',
        'isHomeRec',
        'isSaved',
        'isShowcaseListing',
        'isUndisclosedAddress',
        'isUserClaimingOwner',
        'isUserConfirmedClaim',
        "countryCurrency"
    ]
    
    for keyValue in keyValues:
        try:
            del house[keyValue]
        except:
            pass

    try:
        house["area"] = str(house["area"])+" sqft"
    except:
        pass

    keysList = [
        "rentZestimate",
        "country",
        "taxAssessedValue",
        "daysOnZillow",
        "lotAreaUnit",
        "lotAreaValue"
    ]
    for key in keysList:
        try:
            house[key] = house["hdpData"]["homeInfo"][key]
        except:
            pass

    try:
        # has a dict in it which has a lot data I picked important ones and deleted it
        del house["hdpData"]
    except:
        pass
    
    
    for key in ["zpid","id","addressZipcode"]:
        try:
            house[key]=int(house[key])
        except:
            pass
            
    
    
    

    # pprint.pprint(house)
    print(1, house)
    return house

def getInfo_fromGeneralPage(link, params, pageNum):
    tmppageList = []
    params["searchQueryState"]["pagination"]["currentPage"] = pageNum
    with requests.Session() as s:
        s.headers['User-Agent'] = 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.150 Safari/537.36'
        s.headers["x-requested-session"] = "BE6D8DA620E60010D84B55EB18DC9DC8"
        s.headers["cookie"] = f"JSESSIONID={s.headers['x-requested-session']}"
    response = s.get(f"{link}{urllib.parse.urlencode(params)}")

    # Print the URL used for the request

    # Process the response data
    data = json.dumps(json.loads(response.content), indent=2)
    jsonData = json.loads(data)
    totalPage = jsonData["cat1"]["searchList"]["totalPages"]

    for house in jsonData["cat1"]["searchResults"]["listResults"][:3]:
        try:
            tmppageList.append(house_formatter(house))
            time.sleep(1)
        except:
            tmppageList.append(house_formatter(house))

    return tmppageList

def getInfo_fromDetailedPage(FirstStageList):
    
    pass


def store_toXlsx(dictionaryList, fileName):
    try:
        df_existing = pd.read_excel(fileName)
        print(fileName+" exists")
    except:
        print("File does not exist creating file: "+fileName)
        df_existing = None
        
        
    print(dictionaryList)
    df_new = pd.DataFrame(dictionaryList)
    
    df_combined = pd.concat([df_existing, df_new], ignore_index=True)
    with pd.ExcelWriter(fileName, engine='openpyxl') as writer:
        df_combined.to_excel(writer, index=False, sheet_name='Sheet')
    print("Data is added to", fileName)


print(31)


def test(dictionaryList, fileName):
    notInExcelList=[]
    
    try:
        df_existing = pd.read_excel(fileName)
        print(fileName+" exists")
        
        #calling check for same zpid function because excel file already exists
    #def checkSamesies()
    

        for index, row in pd.DataFrame(dictionaryList).iterrows():
            zpid = int(row['zpid'])
            if zpid in df_existing["zpid"].values:
                print(zpid," in excel")
                continue
            else:
                print(zpid," is not in excel")
                notInExcelList.append(row)
        print("Amount of new entries is:",len(notInExcelList))
        df_new=pd.DataFrame(notInExcelList)
    
    except Exception as m:
        print(m)
        print("File does not exist creating file: "+fileName)
        df_existing = None
        df_new = pd.DataFrame(dictionaryList)
        
    
    
    
    
    
    df_combined = pd.concat([df_existing, df_new], ignore_index=True)
    with pd.ExcelWriter(fileName, engine='openpyxl') as writer:
        df_combined.to_excel(writer, index=False, sheet_name='Sheet')
    print("Data is added to", fileName)


test(getInfo_fromGeneralPage(link, params, 1), "test.xlsx")

31
1 {'propertyStatusCd': 'Single Family', 'rawHomeStatusCd': 'ForSale', 'marketingStatusSimplifiedCd': 'For Sale by Agent', 'zpid': 26827056, 'id': 26827056, 'providerListingId': None, 'imgSrc': 'https://photos.zillowstatic.com/fp/6327ddf2bcf0e27701eed2c42a984d99-p_e.jpg', 'carouselPhotos': [{'url': 'https://photos.zillowstatic.com/fp/6327ddf2bcf0e27701eed2c42a984d99-p_e.jpg'}, {'url': 'https://photos.zillowstatic.com/fp/b8dfeb8c22af76fbd11fb1c38cdb4cfc-p_e.jpg'}, {'url': 'https://photos.zillowstatic.com/fp/ac91d8af1c45571c569a32a2df709079-p_e.jpg'}, {'url': 'https://photos.zillowstatic.com/fp/a3affea6966ce15d613f0271a1a88956-p_e.jpg'}, {'url': 'https://photos.zillowstatic.com/fp/540c16a7e03e22fc5e9d3f3ddc92a942-p_e.jpg'}, {'url': 'https://photos.zillowstatic.com/fp/d96945f12dbbe63413702fd8e8dd0ac6-p_e.jpg'}, {'url': 'https://photos.zillowstatic.com/fp/736062abddbbf3c0e89363385bd95ce3-p_e.jpg'}, {'url': 'https://photos.zillowstatic.com/fp/32bc08a07cc4869a1c39fc17068d94e4-p_e.jpg'}, {'

In [ ]:
import requests
import pprint
import json
import urllib
import openpyxl

import pandas as pd
import pprint
import time

link = 'https://www.zillow.com/search/GetSearchPageState.htm?'
params = {
    "wants": {"cat1": ["listResults"], "cat2": ["total"]},
    "requestId": 5,

    "searchQueryState": {"pagination": {"currentPage": 1}, "usersSearchTerm": "Dallas TX", "mapBounds": {"west": -97.43455125097657, "east": -96.12031174902344, "south": 32.431040510618146, "north": 33.20313651601504}, "regionSelection": [{"regionId": 38128, "regionType": 6}], "isMapVisible": "false", "filterState": {"sortSelection": {"value": "globalrelevanceex"}}, "isListVisible": "true"}

}

#util
def house_formatter(house):

    keyValues = [
        "addressStreet",
        "availabilityDate",
        "badgeInfo",
        'has3DModel',
        'hasAdditionalAttributions',
        "hasImage",
        'hasOpenHouse',
        'hasVideo',
        "unformattedPrice",
        'isFeaturedListing',
        'isHomeRec',
        'isSaved',
        "id",
        'isShowcaseListing',
        'isUndisclosedAddress',
        'isUserClaimingOwner',
        'isUserConfirmedClaim',
        "countryCurrency"
    ]
    
    for keyValue in keyValues:
        try:
            del house[keyValue]
        except:
            pass

    try:
        house["area"] = str(house["area"])+" sqft"
    except:
        pass

    keysList = [
        "rentZestimate",
        "country",
        "taxAssessedValue",
        "lotAreaUnit",
        "lotAreaValue"
    ]
    for key in keysList:
        try:
            house[key] = house["hdpData"]["homeInfo"][key]
        except:
            pass

    try:
        # has a dict in it which has a lot data I picked important ones and deleted it
        del house["hdpData"]
    except:
        pass
    
    
    for key in ["zpid","addressZipcode"]:
        try:
            house[key]=int(house[key])
        except:
            pass
            
    
    
    

    # pprint.pprint(house)
    print("Step 1 is complete",house["zpid"])
    return house

def getInfo_fromGeneralPage(link, params, pageNum,fileName):
    tmppageList = []
    params["searchQueryState"]["pagination"]["currentPage"] = pageNum
    with requests.Session() as s:
        s.headers['User-Agent'] = 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.150 Safari/537.36'
        s.headers["x-requested-session"] = "BE6D8DA620E60010D84B55EB18DC9DC8"
        s.headers["cookie"] = f"JSESSIONID={s.headers['x-requested-session']}"
    response = s.get(f"{link}{urllib.parse.urlencode(params)}")

    # Print the URL used for the request

    # Process the response data
    data = json.dumps(json.loads(response.content), indent=2)
    jsonData = json.loads(data)
    totalPage = jsonData["cat1"]["searchList"]["totalPages"]
    try:
        df_existing = pd.read_excel(fileName)
        print(fileName+" exists")
        for house in jsonData["cat1"]["searchResults"]["listResults"][:100]:
        
        
            
            if int(house["zpid"]) in df_existing["zpid"].values:
                print(house["zpid"]," IN FUCKING excel")
                continue
            else:
                print(house["zpid"]," NO NO NO IN excel")
                try:
                    tmppageList.append(house_formatter(house))
                    time.sleep(0.5)
                except:
                    tmppageList.append(house_formatter(house))
        return tmppageList
    except:
        for house in jsonData["cat1"]["searchResults"]["listResults"][:100]:
            try:
                tmppageList.append(house_formatter(house))
                time.sleep(1)
            except:
                try:
                    tmppageList.append(house_formatter(house))
                except:
                    print("Failed to scrape:",house)
                    pass
        return tmppageList

    







print(31)

# Load the Excel file
def adjustColumnWidth(file_path):
    workbook = openpyxl.load_workbook(file_path)

    # Select the first sheet (you can change the sheet name if needed)
    sheet = workbook.active

    # Iterate through columns and adjust the width based on the length of the first item
    for column in sheet.columns:
        
        column_letter = openpyxl.utils.get_column_letter(column[0].column)  # Get the column letter
        
        first_cell = column[0]  # Get the first cell in the column
        if first_cell.value == "zpid":
            adjusted_width = 12
            sheet.column_dimensions[column_letter].width = adjusted_width
        else:
            # Set the column width based on the length of the first item
            adjusted_width = len(str(first_cell.value)) + 2  # Add a little extra space
            sheet.column_dimensions[column_letter].width = adjusted_width
    workbook.save(file_path)


def test2(dictionaryList, fileName):
    try:
        df_existing = pd.read_excel(fileName)
    except:
        print("File does not exist creating file: "+fileName)
        df_existing=None
        
    df_new=pd.DataFrame(dictionaryList)
    
    df_combined = pd.concat([df_existing, df_new], ignore_index=True)
    
    with pd.ExcelWriter(fileName, engine='openpyxl') as writer:
        df_combined.to_excel(writer, index=False, sheet_name='Sheet')
    print("Data is added to", fileName)
    adjustColumnWidth(fileName)
    

    
        




def getInfo_fromDetailedPage(zpid):
    
    url = f"https://www.zillow.com/graphql/?extensions=%7B%22persistedQuery%22%3A%7B%22version%22%3A1%2C%22sha256Hash%22%3A%2239561ee3168d472db98de0c5da7f57fd29fb76ed7840e064da6347f007d58f90%22%7D%7D&variables=%7B%22zpid%22%3A{zpid}%2C%22contactFormRenderParameter%22%3A%7B%22zpid%22%3A{zpid}%2C%22platform%22%3A%22desktop%22%2C%22isDoubleScroll%22%3Atrue%7D%2C%22skipCFRD%22%3Afalse%7D"

    payload = {}
    headers = {
        'authority': 'www.zillow.com',
        'accept': '*/*',
        'accept-language': 'tr-TR,tr;q=0.9,en-US;q=0.8,en;q=0.7',
        'client-id': 'for-sale-sub-app-browser-client',
        'content-type': 'application/json',
        'cookie': 'x-amz-continuous-deployment-state=AYABeMel6F34shE2U+uoJxVKYJcAPgACAAFEAB1kM2Jsa2Q0azB3azlvai5jbG91ZGZyb250Lm5ldAABRwAVRzA3MjU1NjcyMVRZRFY4RDcyVlpWAAEAAkNEABpDb29raWUAAACAAAAADHIdAf7ZNMOMZspzhAAwaPKe1Y0NsBOVEtgJJOK+Mn3knMRYPruXU6rkggkRBx+oSSVllPRf++vdWWg%2FSFFPAgAAAAAMAAQAAAAAAAAAAAAAAAAAAFhkTvmBZkLp01e%2FXlJJKcn%2F%2F%2F%2F%2FAAAAAQAAAAAAAAAAAAAAAQAAAAzht9umOLxb19t+0NJhSuU7jsXlj51Q3o4XPw8F; zguid=24|%242ee18300-f1c2-4c4c-894d-e95cdb5a8765; zgsession=1|34c3097e-3313-40f4-bc8b-dda201d247e0; zjs_anonymous_id=%222ee18300-f1c2-4c4c-894d-e95cdb5a8765%22; zjs_user_id=null; zg_anonymous_id=%22fba31dc9-7da4-465a-87f0-e352d42726c6%22; _gcl_au=1.1.1962137471.1691074681; DoubleClickSession=true; pxcts=47bbb743-39c1-11ee-a406-4c704d515a67; _pxvid=47bba73a-39c1-11ee-a406-69b061c9f9e6; JSESSIONID=81E99BD7B574C62C2B6996CAFF097408; x-amz-continuous-deployment-state=AYABeOobNKWMlqXk6Opv4KqHZ00APgACAAFEAB1kM2Jsa2Q0azB3azlvai5jbG91ZGZyb250Lm5ldAABRwAVRzA3MjU1NjcyMVRZRFY4RDcyVlpWAAEAAkNEABpDb29raWUAAACAAAAADIPw8Tz1c%2FJkcGJ0sQAw+rocAg2V+6AzJbuvD%2Fa4s+Ylx4br+E+wY2L6P6cJzA68%2Fqwb7RVmdfTZaPwT20EtAgAAAAAMAAQAAAAAAAAAAAAAAAAAANqit%2FMngxR65bjOjBYJVm%2F%2F%2F%2F%2F%2FAAAAAQAAAAAAAAAAAAAAAQAAAAx+hQx+vBn9U086grypTwjpTYN7VgMM3GdrUppTTwjpTYN7VgMM3GdrUppTTwjpTYN7VgMM3GdrUppTTwjpTYN7VgMM3GdrUppTTwjpTYN7VgMM3GdrUppT; search=6|1695876639229%7Crect%3D32.80386902766574%252C-96.68895721435547%252C32.70728894920813%252C-96.85323715209961%26rid%3D38128%26disp%3Dmap%26mdm%3Dauto%26sort%3Dpriorityscore%26listPriceActive%3D1%26fs%3D1%26fr%3D0%26mmm%3D0%26rs%3D0%26ah%3D0%26singlestory%3D0%26abo%3D0%26garage%3D0%26pool%3D0%26ac%3D0%26waterfront%3D0%26finished%3D0%26unfinished%3D0%26cityview%3D0%26mountainview%3D0%26parkview%3D0%26waterview%3D0%26hoadata%3D1%263dhome%3D0%26commuteMode%3Ddriving%26commuteTimeOfDay%3Dnow%09%0938128%09%7B%22isList%22%3Atrue%2C%22isMap%22%3Atrue%7D%09%09%09%09%09; AWSALB=UT3+qcYR3tHuczo/TbzN8YUy1B0/uBnd1W1eCwJmlP4fdIlrjUnSL0VikXGyr5hCjVRJPoZVqpN1joR4OVqQxhClYX9Jfm9NllqrJ5hEwtA5Zm73ARi0S8rV4i+E; AWSALBCORS=UT3+qcYR3tHuczo/TbzN8YUy1B0/uBnd1W1eCwJmlP4fdIlrjUnSL0VikXGyr5hCjVRJPoZVqpN1joR4OVqQxhClYX9Jfm9NllqrJ5hEwtA5Zm73ARi0S8rV4i+E; x-amz-continuous-deployment-state=AYABeIs8bDpDqQvTOLxmullpMEoAPgACAAFEAB1kM2Jsa2Q0azB3azlvai5jbG91ZGZyb250Lm5ldAABRwAVRzA3MjU1NjcyMVRZRFY4RDcyVlpWAAEAAkNEABpDb29raWUAAACAAAAADLqrfBR3ecBLDhYnbgAwZtZr8viUp3066btA1CybuVxFoUj3kYCq9JKc46gmzh47FTKYM4PIfZ5Vep1OqhhkAgAAAAAMAAQAAAAAAAAAAAAAAAAAANLcxYezdf26FjHJLrj9XDH%2F%2F%2F%2F%2FAAAAAQAAAAAAAAAAAAAAAQAAAAyIYroULD24T6PCx5cudKnJavjLoGK4W1ukGYks; AWSALB=CmB/8DLiDb1ycFVjeKynVX6F8ahPkMrM+ZR3RoL8wCsSAh1Xp7ELPH/319qgZl8Y6wzpbYkwg3cRX0ab6OvPb1Lew9o179ZvkT51W3UGGIqDdVAIJFOMMVCD3b9r; AWSALBCORS=CmB/8DLiDb1ycFVjeKynVX6F8ahPkMrM+ZR3RoL8wCsSAh1Xp7ELPH/319qgZl8Y6wzpbYkwg3cRX0ab6OvPb1Lew9o179ZvkT51W3UGGIqDdVAIJFOMMVCD3b9r',
        'referer': f'https://www.zillow.com/homedetails/{zpid}_zpid/',
        'sec-ch-ua': '"Not.A/Brand";v="8", "Chromium";v="114", "Opera GX";v="100"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"Windows"',
        'sec-fetch-dest': 'empty',
        'sec-fetch-mode': 'cors',
        'sec-fetch-site': 'same-origin',
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36 OPR/100.0.0.0',
        }

    response = requests.request("GET", url, headers=headers, data=payload)

    json=response.json()
    text=json["data"]["property"]
    listingInfo=json["data"]["property"]["attributionInfo"]
    
    data_dict = {
    "Agent Name": listingInfo["agentName"],
    "Agent Phone Number": listingInfo["agentPhoneNumber"],
    "Agent License Number": listingInfo["agentLicenseNumber"],
    "Broker Name":listingInfo["brokerName"],
    "Broker Phone Number": listingInfo["brokerPhoneNumber"],
    "Time on Zillow": text["timeOnZillow"],
    "Page View Count": text["pageViewCount"],
    "Favorite Count": text["favoriteCount"],
    }
    
    return data_dict

fileName="zillowtest.xlsx"
bigData=getInfo_fromGeneralPage(link, params, 1,fileName)

def control(zpid):
    url = f"https://www.zillow.com/graphql/?extensions=%7B%22persistedQuery%22%3A%7B%22version%22%3A1%2C%22sha256Hash%22%3A%2297e1ac1e2f4ccd802cecec9058f0fcab998225b21a0c62fa3234635ff2bd53cd%22%7D%7D&variables=%7B%22zpid%22%3A{zpid}%2C%22platform%22%3A%22DESKTOP_WEB%22%2C%22formType%22%3A%22OPAQUE%22%2C%22contactFormRenderParameter%22%3A%7B%22zpid%22%3A{zpid}%2C%22platform%22%3A%22desktop%22%2C%22isDoubleScroll%22%3Atrue%7D%2C%22skipCFRD%22%3Afalse%7D"
    payload = {}
    headers = {
        'authority': 'www.zillow.com',
        'accept': '*/*',
        'accept-language': 'tr-TR,tr;q=0.9,en-US;q=0.8,en;q=0.7',
        'client-id': 'for-sale-sub-app-browser-client',
        'content-type': 'application/json',
        'cookie': 'x-amz-continuous-deployment-state=AYABeMel6F34shE2U+uoJxVKYJcAPgACAAFEAB1kM2Jsa2Q0azB3azlvai5jbG91ZGZyb250Lm5ldAABRwAVRzA3MjU1NjcyMVRZRFY4RDcyVlpWAAEAAkNEABpDb29raWUAAACAAAAADHIdAf7ZNMOMZspzhAAwaPKe1Y0NsBOVEtgJJOK+Mn3knMRYPruXU6rkggkRBx+oSSVllPRf++vdWWg%2FSFFPAgAAAAAMAAQAAAAAAAAAAAAAAAAAAFhkTvmBZkLp01e%2FXlJJKcn%2F%2F%2F%2F%2FAAAAAQAAAAAAAAAAAAAAAQAAAAzht9umOLxb19t+0NJhSuU7jsXlj51Q3o4XPw8F; zguid=24|%242ee18300-f1c2-4c4c-894d-e95cdb5a8765; zgsession=1|34c3097e-3313-40f4-bc8b-dda201d247e0; zjs_anonymous_id=%222ee18300-f1c2-4c4c-894d-e95cdb5a8765%22; zjs_user_id=null; zg_anonymous_id=%22fba31dc9-7da4-465a-87f0-e352d42726c6%22; _gcl_au=1.1.1962137471.1691074681; DoubleClickSession=true; pxcts=47bbb743-39c1-11ee-a406-4c704d515a67; _pxvid=47bba73a-39c1-11ee-a406-69b061c9f9e6; JSESSIONID=81E99BD7B574C62C2B6996CAFF097408; x-amz-continuous-deployment-state=AYABeOobNKWMlqXk6Opv4KqHZ00APgACAAFEAB1kM2Jsa2Q0azB3azlvai5jbG91ZGZyb250Lm5ldAABRwAVRzA3MjU1NjcyMVRZRFY4RDcyVlpWAAEAAkNEABpDb29raWUAAACAAAAADIPw8Tz1c%2FJkcGJ0sQAw+rocAg2V+6AzJbuvD%2Fa4s+Ylx4br+E+wY2L6P6cJzA68%2Fqwb7RVmdfTZaPwT20EtAgAAAAAMAAQAAAAAAAAAAAAAAAAAANqit%2FMngxR65bjOjBYJVm%2F%2F%2F%2F%2F%2FAAAAAQAAAAAAAAAAAAAAAQAAAAx+hQx+vBn9U086grypTwjpTYN7VgMM3GdrUppTTwjpTYN7VgMM3GdrUppTTwjpTYN7VgMM3GdrUppTTwjpTYN7VgMM3GdrUppTTwjpTYN7VgMM3GdrUppT; search=6|1695876639229%7Crect%3D32.80386902766574%252C-96.68895721435547%252C32.70728894920813%252C-96.85323715209961%26rid%3D38128%26disp%3Dmap%26mdm%3Dauto%26sort%3Dpriorityscore%26listPriceActive%3D1%26fs%3D1%26fr%3D0%26mmm%3D0%26rs%3D0%26ah%3D0%26singlestory%3D0%26abo%3D0%26garage%3D0%26pool%3D0%26ac%3D0%26waterfront%3D0%26finished%3D0%26unfinished%3D0%26cityview%3D0%26mountainview%3D0%26parkview%3D0%26waterview%3D0%26hoadata%3D1%263dhome%3D0%26commuteMode%3Ddriving%26commuteTimeOfDay%3Dnow%09%0938128%09%7B%22isList%22%3Atrue%2C%22isMap%22%3Atrue%7D%09%09%09%09%09; AWSALB=UT3+qcYR3tHuczo/TbzN8YUy1B0/uBnd1W1eCwJmlP4fdIlrjUnSL0VikXGyr5hCjVRJPoZVqpN1joR4OVqQxhClYX9Jfm9NllqrJ5hEwtA5Zm73ARi0S8rV4i+E; AWSALBCORS=UT3+qcYR3tHuczo/TbzN8YUy1B0/uBnd1W1eCwJmlP4fdIlrjUnSL0VikXGyr5hCjVRJPoZVqpN1joR4OVqQxhClYX9Jfm9NllqrJ5hEwtA5Zm73ARi0S8rV4i+E; x-amz-continuous-deployment-state=AYABeIs8bDpDqQvTOLxmullpMEoAPgACAAFEAB1kM2Jsa2Q0azB3azlvai5jbG91ZGZyb250Lm5ldAABRwAVRzA3MjU1NjcyMVRZRFY4RDcyVlpWAAEAAkNEABpDb29raWUAAACAAAAADLqrfBR3ecBLDhYnbgAwZtZr8viUp3066btA1CybuVxFoUj3kYCq9JKc46gmzh47FTKYM4PIfZ5Vep1OqhhkAgAAAAAMAAQAAAAAAAAAAAAAAAAAANLcxYezdf26FjHJLrj9XDH%2F%2F%2F%2F%2FAAAAAQAAAAAAAAAAAAAAAQAAAAyIYroULD24T6PCx5cudKnJavjLoGK4W1ukGYks; AWSALB=CmB/8DLiDb1ycFVjeKynVX6F8ahPkMrM+ZR3RoL8wCsSAh1Xp7ELPH/319qgZl8Y6wzpbYkwg3cRX0ab6OvPb1Lew9o179ZvkT51W3UGGIqDdVAIJFOMMVCD3b9r; AWSALBCORS=CmB/8DLiDb1ycFVjeKynVX6F8ahPkMrM+ZR3RoL8wCsSAh1Xp7ELPH/319qgZl8Y6wzpbYkwg3cRX0ab6OvPb1Lew9o179ZvkT51W3UGGIqDdVAIJFOMMVCD3b9r',
        'referer': f'https://www.zillow.com/homedetails/{zpid}_zpid/',
        'sec-ch-ua': '"Not.A/Brand";v="8", "Chromium";v="114", "Opera GX";v="100"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"Windows"',
        'sec-fetch-dest': 'empty',
        'sec-fetch-mode': 'cors',
        'sec-fetch-site': 'same-origin',
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36 OPR/100.0.0.0',
        }

    response = requests.request("GET", url, headers=headers, data=payload)
    json=response.json()
    text=json["data"]["property"]
    listingInfo=json["data"]["property"]["attributionInfo"]
    
    data_dict = {
    "Agent Name": listingInfo["agentName"],
    "Agent Phone Number": listingInfo["agentPhoneNumber"],
    "Agent License Number": listingInfo["agentLicenseNumber"],
    "Broker Name":listingInfo["brokerName"],
    "Broker Phone Number": listingInfo["brokerPhoneNumber"],
    "Time on Zillow": text["timeOnZillow"],
    "Page View Count": text["pageViewCount"],
    "Favorite Count": text["favoriteCount"],
    }
    print("Step 2 is complete",zpid)
    return data_dict


for listing in bigData:
    try:
        listing.update(control(listing["zpid"]))
    except:
        print("Error at:",listing)

test2(bigData, fileName)



In [22]:
zpid2=26810709
def control2(zpid):
    url = f"https://www.zillow.com/graphql/?extensions=%7B%22persistedQuery%22%3A%7B%22version%22%3A1%2C%22sha256Hash%22%3A%2297e1ac1e2f4ccd802cecec9058f0fcab998225b21a0c62fa3234635ff2bd53cd%22%7D%7D&variables=%7B%22zpid%22%3A{zpid}%2C%22platform%22%3A%22DESKTOP_WEB%22%2C%22formType%22%3A%22OPAQUE%22%2C%22contactFormRenderParameter%22%3A%7B%22zpid%22%3A{zpid}%2C%22platform%22%3A%22desktop%22%2C%22isDoubleScroll%22%3Atrue%7D%2C%22skipCFRD%22%3Afalse%7D"
    payload = {}
    headers = {
        'authority': 'www.zillow.com',
        'accept': '*/*',
        'accept-language': 'tr-TR,tr;q=0.9,en-US;q=0.8,en;q=0.7',
        'client-id': 'for-sale-sub-app-browser-client',
        'content-type': 'application/json',
        'cookie': 'x-amz-continuous-deployment-state=AYABeMel6F34shE2U+uoJxVKYJcAPgACAAFEAB1kM2Jsa2Q0azB3azlvai5jbG91ZGZyb250Lm5ldAABRwAVRzA3MjU1NjcyMVRZRFY4RDcyVlpWAAEAAkNEABpDb29raWUAAACAAAAADHIdAf7ZNMOMZspzhAAwaPKe1Y0NsBOVEtgJJOK+Mn3knMRYPruXU6rkggkRBx+oSSVllPRf++vdWWg%2FSFFPAgAAAAAMAAQAAAAAAAAAAAAAAAAAAFhkTvmBZkLp01e%2FXlJJKcn%2F%2F%2F%2F%2FAAAAAQAAAAAAAAAAAAAAAQAAAAzht9umOLxb19t+0NJhSuU7jsXlj51Q3o4XPw8F; zguid=24|%242ee18300-f1c2-4c4c-894d-e95cdb5a8765; zgsession=1|34c3097e-3313-40f4-bc8b-dda201d247e0; zjs_anonymous_id=%222ee18300-f1c2-4c4c-894d-e95cdb5a8765%22; zjs_user_id=null; zg_anonymous_id=%22fba31dc9-7da4-465a-87f0-e352d42726c6%22; _gcl_au=1.1.1962137471.1691074681; DoubleClickSession=true; pxcts=47bbb743-39c1-11ee-a406-4c704d515a67; _pxvid=47bba73a-39c1-11ee-a406-69b061c9f9e6; JSESSIONID=81E99BD7B574C62C2B6996CAFF097408; x-amz-continuous-deployment-state=AYABeOobNKWMlqXk6Opv4KqHZ00APgACAAFEAB1kM2Jsa2Q0azB3azlvai5jbG91ZGZyb250Lm5ldAABRwAVRzA3MjU1NjcyMVRZRFY4RDcyVlpWAAEAAkNEABpDb29raWUAAACAAAAADIPw8Tz1c%2FJkcGJ0sQAw+rocAg2V+6AzJbuvD%2Fa4s+Ylx4br+E+wY2L6P6cJzA68%2Fqwb7RVmdfTZaPwT20EtAgAAAAAMAAQAAAAAAAAAAAAAAAAAANqit%2FMngxR65bjOjBYJVm%2F%2F%2F%2F%2F%2FAAAAAQAAAAAAAAAAAAAAAQAAAAx+hQx+vBn9U086grypTwjpTYN7VgMM3GdrUppTTwjpTYN7VgMM3GdrUppTTwjpTYN7VgMM3GdrUppTTwjpTYN7VgMM3GdrUppTTwjpTYN7VgMM3GdrUppT; search=6|1695876639229%7Crect%3D32.80386902766574%252C-96.68895721435547%252C32.70728894920813%252C-96.85323715209961%26rid%3D38128%26disp%3Dmap%26mdm%3Dauto%26sort%3Dpriorityscore%26listPriceActive%3D1%26fs%3D1%26fr%3D0%26mmm%3D0%26rs%3D0%26ah%3D0%26singlestory%3D0%26abo%3D0%26garage%3D0%26pool%3D0%26ac%3D0%26waterfront%3D0%26finished%3D0%26unfinished%3D0%26cityview%3D0%26mountainview%3D0%26parkview%3D0%26waterview%3D0%26hoadata%3D1%263dhome%3D0%26commuteMode%3Ddriving%26commuteTimeOfDay%3Dnow%09%0938128%09%7B%22isList%22%3Atrue%2C%22isMap%22%3Atrue%7D%09%09%09%09%09; AWSALB=UT3+qcYR3tHuczo/TbzN8YUy1B0/uBnd1W1eCwJmlP4fdIlrjUnSL0VikXGyr5hCjVRJPoZVqpN1joR4OVqQxhClYX9Jfm9NllqrJ5hEwtA5Zm73ARi0S8rV4i+E; AWSALBCORS=UT3+qcYR3tHuczo/TbzN8YUy1B0/uBnd1W1eCwJmlP4fdIlrjUnSL0VikXGyr5hCjVRJPoZVqpN1joR4OVqQxhClYX9Jfm9NllqrJ5hEwtA5Zm73ARi0S8rV4i+E; x-amz-continuous-deployment-state=AYABeIs8bDpDqQvTOLxmullpMEoAPgACAAFEAB1kM2Jsa2Q0azB3azlvai5jbG91ZGZyb250Lm5ldAABRwAVRzA3MjU1NjcyMVRZRFY4RDcyVlpWAAEAAkNEABpDb29raWUAAACAAAAADLqrfBR3ecBLDhYnbgAwZtZr8viUp3066btA1CybuVxFoUj3kYCq9JKc46gmzh47FTKYM4PIfZ5Vep1OqhhkAgAAAAAMAAQAAAAAAAAAAAAAAAAAANLcxYezdf26FjHJLrj9XDH%2F%2F%2F%2F%2FAAAAAQAAAAAAAAAAAAAAAQAAAAyIYroULD24T6PCx5cudKnJavjLoGK4W1ukGYks; AWSALB=CmB/8DLiDb1ycFVjeKynVX6F8ahPkMrM+ZR3RoL8wCsSAh1Xp7ELPH/319qgZl8Y6wzpbYkwg3cRX0ab6OvPb1Lew9o179ZvkT51W3UGGIqDdVAIJFOMMVCD3b9r; AWSALBCORS=CmB/8DLiDb1ycFVjeKynVX6F8ahPkMrM+ZR3RoL8wCsSAh1Xp7ELPH/319qgZl8Y6wzpbYkwg3cRX0ab6OvPb1Lew9o179ZvkT51W3UGGIqDdVAIJFOMMVCD3b9r',
        'referer': f'https://www.zillow.com/homedetails/{zpid}_zpid/',
        'sec-ch-ua': '"Not.A/Brand";v="8", "Chromium";v="114", "Opera GX";v="100"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"Windows"',
        'sec-fetch-dest': 'empty',
        'sec-fetch-mode': 'cors',
        'sec-fetch-site': 'same-origin',
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36 OPR/100.0.0.0',
        }

    response = requests.request("GET", url, headers=headers, data=payload)
    json=response.json()
    text=json["data"]["property"]
    listingInfo=json["data"]["property"]["attributionInfo"]
    
    data_dict = {
    "Agent Name": listingInfo["agentName"],
    "Agent Phone Number": listingInfo["agentPhoneNumber"],
    "Agent License Number": listingInfo["agentLicenseNumber"],
    "Broker Name":listingInfo["brokerName"],
    "Broker Phone Number": listingInfo["brokerPhoneNumber"],
    "Description":text["description"],
    "Time on Zillow": text["timeOnZillow"],
    "Page View Count": text["pageViewCount"],
    "Favorite Count": text["favoriteCount"],
    }
    print("Step 2 is complete",zpid)
    return data_dict

control2(zpid2)

Step 2 is complete 26810709


{'Agent Name': 'Maggie Dieter',
 'Agent Phone Number': None,
 'Agent License Number': '0727229',
 'Broker Name': 'Keller Williams Central',
 'Broker Phone Number': '469-467-7755',
 'Description': "You won't want to miss this amazing 2-bedroom, 2-bath end-unit condo with a private courtyard in West Dallas! Bright, open-concept first floor with wood-burning fireplace and updated flooring. The kitchen features a breakfast bar, black and stainless-steel appliances, a pantry, and ample cabinet space. The primary bedroom is located on the main floor, offers access to the private courtyard, and has his & her closets, and the ensuite bath features two vanities and a tile shower-tub combo. The 2nd bedroom and bath are upstairs, perfect for a roommate, child, or multi-generational living. Walk outside your door & you are at the pool! Just 10 minutes away from Preston Hollow, near top-ranked private schools, & close to everything! One covered parking space is included. Easy access to I-35, 635, D

In [12]:
zpid=69676592
zpid1=26697685
def control(zpid):
    url = f"https://www.zillow.com/graphql/?extensions=%7B%22persistedQuery%22%3A%7B%22version%22%3A1%2C%22sha256Hash%22%3A%2297e1ac1e2f4ccd802cecec9058f0fcab998225b21a0c62fa3234635ff2bd53cd%22%7D%7D&variables=%7B%22zpid%22%3A{zpid}%2C%22platform%22%3A%22DESKTOP_WEB%22%2C%22formType%22%3A%22OPAQUE%22%2C%22contactFormRenderParameter%22%3A%7B%22zpid%22%3A{zpid}%2C%22platform%22%3A%22desktop%22%2C%22isDoubleScroll%22%3Atrue%7D%2C%22skipCFRD%22%3Afalse%7D"
    payload = {}
    headers = {
        'authority': 'www.zillow.com',
        'accept': '*/*',
        'accept-language': 'tr-TR,tr;q=0.9,en-US;q=0.8,en;q=0.7',
        'client-id': 'for-sale-sub-app-browser-client',
        'content-type': 'application/json',
        'cookie': 'x-amz-continuous-deployment-state=AYABeMel6F34shE2U+uoJxVKYJcAPgACAAFEAB1kM2Jsa2Q0azB3azlvai5jbG91ZGZyb250Lm5ldAABRwAVRzA3MjU1NjcyMVRZRFY4RDcyVlpWAAEAAkNEABpDb29raWUAAACAAAAADHIdAf7ZNMOMZspzhAAwaPKe1Y0NsBOVEtgJJOK+Mn3knMRYPruXU6rkggkRBx+oSSVllPRf++vdWWg%2FSFFPAgAAAAAMAAQAAAAAAAAAAAAAAAAAAFhkTvmBZkLp01e%2FXlJJKcn%2F%2F%2F%2F%2FAAAAAQAAAAAAAAAAAAAAAQAAAAzht9umOLxb19t+0NJhSuU7jsXlj51Q3o4XPw8F; zguid=24|%242ee18300-f1c2-4c4c-894d-e95cdb5a8765; zgsession=1|34c3097e-3313-40f4-bc8b-dda201d247e0; zjs_anonymous_id=%222ee18300-f1c2-4c4c-894d-e95cdb5a8765%22; zjs_user_id=null; zg_anonymous_id=%22fba31dc9-7da4-465a-87f0-e352d42726c6%22; _gcl_au=1.1.1962137471.1691074681; DoubleClickSession=true; pxcts=47bbb743-39c1-11ee-a406-4c704d515a67; _pxvid=47bba73a-39c1-11ee-a406-69b061c9f9e6; JSESSIONID=81E99BD7B574C62C2B6996CAFF097408; x-amz-continuous-deployment-state=AYABeOobNKWMlqXk6Opv4KqHZ00APgACAAFEAB1kM2Jsa2Q0azB3azlvai5jbG91ZGZyb250Lm5ldAABRwAVRzA3MjU1NjcyMVRZRFY4RDcyVlpWAAEAAkNEABpDb29raWUAAACAAAAADIPw8Tz1c%2FJkcGJ0sQAw+rocAg2V+6AzJbuvD%2Fa4s+Ylx4br+E+wY2L6P6cJzA68%2Fqwb7RVmdfTZaPwT20EtAgAAAAAMAAQAAAAAAAAAAAAAAAAAANqit%2FMngxR65bjOjBYJVm%2F%2F%2F%2F%2F%2FAAAAAQAAAAAAAAAAAAAAAQAAAAx+hQx+vBn9U086grypTwjpTYN7VgMM3GdrUppTTwjpTYN7VgMM3GdrUppTTwjpTYN7VgMM3GdrUppTTwjpTYN7VgMM3GdrUppTTwjpTYN7VgMM3GdrUppT; search=6|1695876639229%7Crect%3D32.80386902766574%252C-96.68895721435547%252C32.70728894920813%252C-96.85323715209961%26rid%3D38128%26disp%3Dmap%26mdm%3Dauto%26sort%3Dpriorityscore%26listPriceActive%3D1%26fs%3D1%26fr%3D0%26mmm%3D0%26rs%3D0%26ah%3D0%26singlestory%3D0%26abo%3D0%26garage%3D0%26pool%3D0%26ac%3D0%26waterfront%3D0%26finished%3D0%26unfinished%3D0%26cityview%3D0%26mountainview%3D0%26parkview%3D0%26waterview%3D0%26hoadata%3D1%263dhome%3D0%26commuteMode%3Ddriving%26commuteTimeOfDay%3Dnow%09%0938128%09%7B%22isList%22%3Atrue%2C%22isMap%22%3Atrue%7D%09%09%09%09%09; AWSALB=UT3+qcYR3tHuczo/TbzN8YUy1B0/uBnd1W1eCwJmlP4fdIlrjUnSL0VikXGyr5hCjVRJPoZVqpN1joR4OVqQxhClYX9Jfm9NllqrJ5hEwtA5Zm73ARi0S8rV4i+E; AWSALBCORS=UT3+qcYR3tHuczo/TbzN8YUy1B0/uBnd1W1eCwJmlP4fdIlrjUnSL0VikXGyr5hCjVRJPoZVqpN1joR4OVqQxhClYX9Jfm9NllqrJ5hEwtA5Zm73ARi0S8rV4i+E; x-amz-continuous-deployment-state=AYABeIs8bDpDqQvTOLxmullpMEoAPgACAAFEAB1kM2Jsa2Q0azB3azlvai5jbG91ZGZyb250Lm5ldAABRwAVRzA3MjU1NjcyMVRZRFY4RDcyVlpWAAEAAkNEABpDb29raWUAAACAAAAADLqrfBR3ecBLDhYnbgAwZtZr8viUp3066btA1CybuVxFoUj3kYCq9JKc46gmzh47FTKYM4PIfZ5Vep1OqhhkAgAAAAAMAAQAAAAAAAAAAAAAAAAAANLcxYezdf26FjHJLrj9XDH%2F%2F%2F%2F%2FAAAAAQAAAAAAAAAAAAAAAQAAAAyIYroULD24T6PCx5cudKnJavjLoGK4W1ukGYks; AWSALB=CmB/8DLiDb1ycFVjeKynVX6F8ahPkMrM+ZR3RoL8wCsSAh1Xp7ELPH/319qgZl8Y6wzpbYkwg3cRX0ab6OvPb1Lew9o179ZvkT51W3UGGIqDdVAIJFOMMVCD3b9r; AWSALBCORS=CmB/8DLiDb1ycFVjeKynVX6F8ahPkMrM+ZR3RoL8wCsSAh1Xp7ELPH/319qgZl8Y6wzpbYkwg3cRX0ab6OvPb1Lew9o179ZvkT51W3UGGIqDdVAIJFOMMVCD3b9r',
        'referer': f'https://www.zillow.com/homedetails/{zpid}_zpid/',
        'sec-ch-ua': '"Not.A/Brand";v="8", "Chromium";v="114", "Opera GX";v="100"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"Windows"',
        'sec-fetch-dest': 'empty',
        'sec-fetch-mode': 'cors',
        'sec-fetch-site': 'same-origin',
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36 OPR/100.0.0.0',
        }

    response = requests.request("GET", url, headers=headers, data=payload)
    json=response.json()
    text=json["data"]["property"]
    listingInfo=json["data"]["property"]["attributionInfo"]
    
    data_dict = {
    "Agent Name": listingInfo["agentName"],
    "Agent Phone Number": listingInfo["agentPhoneNumber"],
    "Agent License Number": listingInfo["agentLicenseNumber"],
    "Broker Name":listingInfo["brokerName"],
    "Broker Phone Number": listingInfo["brokerPhoneNumber"],
    "Time on Zillow": text["timeOnZillow"],
    "Page View Count": text["pageViewCount"],
    "Favorite Count": text["favoriteCount"],
    }
    
    return data_dict
control(zpid1)


{'Agent Name': 'Jorge Mata',
 'Agent Phone Number': None,
 'Agent License Number': '0734885',
 'Broker Name': 'OnDemand Realty',
 'Broker Phone Number': '214-766-5833',
 'Time on Zillow': '17 days',
 'Page View Count': 2380,
 'Favorite Count': 181}

In [7]:
import requests
import json

url = "https://www.zillow.com/graphql/?extensions=%7B%22persistedQuery%22%3A%7B%22version%22%3A1%2C%22sha256Hash%22%3A%2297e1ac1e2f4ccd802cecec9058f0fcab998225b21a0c62fa3234635ff2bd53cd%22%7D%7D&variables=%7B%22zpid%22%3A69676592%2C%22platform%22%3A%22DESKTOP_WEB%22%2C%22formType%22%3A%22OPAQUE%22%2C%22contactFormRenderParameter%22%3A%7B%22zpid%22%3A69676592%2C%22platform%22%3A%22desktop%22%2C%22isDoubleScroll%22%3Atrue%7D%2C%22skipCFRD%22%3Afalse%7D"

payload = {}
headers = {
  'authority': 'www.zillow.com',
  'accept': '*/*',
  'accept-language': 'tr-TR,tr;q=0.9,en-US;q=0.8,en;q=0.7',
  'client-id': 'for-sale-sub-app-browser-client',
  'content-type': 'application/json',
  'cookie': 'x-amz-continuous-deployment-state=AYABeIdR5wen9xVPECRhPRKbf3AAPgACAAFEAB1kM2Jsa2Q0azB3azlvai5jbG91ZGZyb250Lm5ldAABRwAVRzA3MjU1NjcyMVRZRFY4RDcyVlpWAAEAAkNEABpDb29raWUAAACAAAAADNrY0AG3Qu0PD6Jx0wAwrj2pKiLdaBcCZQF+aZdS4arNm6K3+IfW+2xxukIhbNv+k70jBFH8xQvgKRu5hmTRAgAAAAAMAAQAAAAAAAAAAAAAAAAAAO8fKLNph2Zoxqk22UXHht3%2F%2F%2F%2F%2FAAAAAQAAAAAAAAAAAAAAAQAAAAwmsDc%2FhlTH7vQOvC1TzaZtf3Z5CTRfh%2F2VwRZI; zguid=24|%242ee18300-f1c2-4c4c-894d-e95cdb5a8765; zgsession=1|34c3097e-3313-40f4-bc8b-dda201d247e0; zjs_anonymous_id=%222ee18300-f1c2-4c4c-894d-e95cdb5a8765%22; zjs_user_id=null; zg_anonymous_id=%22fba31dc9-7da4-465a-87f0-e352d42726c6%22; _gcl_au=1.1.1962137471.1691074681; DoubleClickSession=true; pxcts=47bbb743-39c1-11ee-a406-4c704d515a67; _pxvid=47bba73a-39c1-11ee-a406-69b061c9f9e6; JSESSIONID=A48B027442F89CDEDED142118AF717E5; x-amz-continuous-deployment-state=AYABePbhGjZO+ADrTnDbCSQW63UAPgACAAFEAB1kM2Jsa2Q0azB3azlvai5jbG91ZGZyb250Lm5ldAABRwAVRzA3MjU1NjcyMVRZRFY4RDcyVlpWAAEAAkNEABpDb29raWUAAACAAAAADMJzWZQpcPZr1xEzQQAwqHfKyeJJLpRokRdofc4kVkMFu3b89bfZvnO25pZaJdtSAe%2FQ9mxXPaGn0zVfLc7YAgAAAAAMAAQAAAAAAAAAAAAAAAAAANBh6KtedpvZ8Ce53ejcZ77%2F%2F%2F%2F%2FAAAAAQAAAAAAAAAAAAAAAQAAAAwie0lWuRCWkEx0lHigxL2mh+TnHimdLYipPoEDHimdLYipPoEDHimdLYipPoEDHimdLYipPoEDHimdLYipPoEDHimdLYipPoEDHimdLYipPoEDHimdLYipPoED; AWSALB=pKf5ObkqYGw9LrkJUiIJvZf6okeSrNstDkSvMowvLifbivJQxstq84ufzLlIGNyRPw/8cvjrl+Ra6Do3ZSj3qvJH6oIwxk1trnQNVh1xFRPtdGQn7/DZnlVeJ/pF; AWSALBCORS=pKf5ObkqYGw9LrkJUiIJvZf6okeSrNstDkSvMowvLifbivJQxstq84ufzLlIGNyRPw/8cvjrl+Ra6Do3ZSj3qvJH6oIwxk1trnQNVh1xFRPtdGQn7/DZnlVeJ/pF; search=6|1699516717936%7C%09%0938128%09%7B%22isList%22%3Atrue%2C%22isMap%22%3Afalse%7D%09%09%09%09%09; x-amz-continuous-deployment-state=AYABeKJW0u+69aaRz0+OB2lTEOsAPgACAAFEAB1kM2Jsa2Q0azB3azlvai5jbG91ZGZyb250Lm5ldAABRwAVRzA3MjU1NjcyMVRZRFY4RDcyVlpWAAEAAkNEABpDb29raWUAAACAAAAADHUO4eDDQFS%2F4HiqugAw59q9bN0y9QO9uNsC9Z%2F21qrU3rnoeHY9w3OKLDHArf+knclSANuKDmW+lC5%2F0XGTAgAAAAAMAAQAAAAAAAAAAAAAAAAAAJjeCcmXeHLFWdEv1yUTZO3%2F%2F%2F%2F%2FAAAAAQAAAAAAAAAAAAAAAQAAAAwnIyEjnPi%2FKsNYx3%2Fkd1aIl972ejDl3RUrW1xq',
  'referer': 'https://www.zillow.com/homedetails/4830-Cedar-Springs-Rd-APT-1-Dallas-TX-75219/69676592_zpid/',
  'sec-ch-ua': '"Chromium";v="116", "Not)A;Brand";v="24", "Opera GX";v="102"',
  'sec-ch-ua-mobile': '?0',
  'sec-ch-ua-platform': '"Windows"',
  'sec-fetch-dest': 'empty',
  'sec-fetch-mode': 'cors',
  'sec-fetch-site': 'same-origin',
  'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36 OPR/102.0.0.0'
}

response = requests.request("GET", url, headers=headers, data=payload)

print(response.text)


{"errors":[{"message":"Expected a value of type \"RESORoomType\" but received: \"HalfBath\"","path":["property","resoFacts","rooms",2,"roomType"],"extensions":{"code":"INTERNAL_SERVER_ERROR","serviceName":"gdp","exception":{"message":"Expected a value of type \"RESORoomType\" but received: \"HalfBath\"","locations":[{"line":1,"column":4231}],"path":["property","resoFacts","rooms",2,"roomType"]}}},{"message":"Expected a value of type \"RESORoomType\" but received: \"FullBath\"","path":["property","resoFacts","rooms",6,"roomType"],"extensions":{"code":"INTERNAL_SERVER_ERROR","serviceName":"gdp","exception":{"message":"Expected a value of type \"RESORoomType\" but received: \"FullBath\"","locations":[{"line":1,"column":4231}],"path":["property","resoFacts","rooms",6,"roomType"]}}},{"message":"Expected a value of type \"RESORoomType\" but received: \"FullBath\"","path":["property","resoFacts","rooms",7,"roomType"],"extensions":{"code":"INTERNAL_SERVER_ERROR","serviceName":"gdp","exception"

In [29]:
import requests
zpid=41068092
def getInfo_fromDetailedPage(zpid):
    url = f"https://www.zillow.com/graphql/?extensions=%7B%22persistedQuery%22%3A%7B%22version%22%3A1%2C%22sha256Hash%22%3A%2239561ee3168d472db98de0c5da7f57fd29fb76ed7840e064da6347f007d58f90%22%7D%7D&variables=%7B%22zpid%22%3A{zpid}%2C%22contactFormRenderParameter%22%3A%7B%22zpid%22%3A{zpid}%2C%22platform%22%3A%22desktop%22%2C%22isDoubleScroll%22%3Atrue%7D%2C%22skipCFRD%22%3Afalse%7D"

    payload = {}
    headers = {
        'authority': 'www.zillow.com',
        'accept': '*/*',
        'accept-language': 'tr-TR,tr;q=0.9,en-US;q=0.8,en;q=0.7',
        'client-id': 'for-sale-sub-app-browser-client',
        'content-type': 'application/json',
        'cookie': 'x-amz-continuous-deployment-state=AYABeMel6F34shE2U+uoJxVKYJcAPgACAAFEAB1kM2Jsa2Q0azB3azlvai5jbG91ZGZyb250Lm5ldAABRwAVRzA3MjU1NjcyMVRZRFY4RDcyVlpWAAEAAkNEABpDb29raWUAAACAAAAADHIdAf7ZNMOMZspzhAAwaPKe1Y0NsBOVEtgJJOK+Mn3knMRYPruXU6rkggkRBx+oSSVllPRf++vdWWg%2FSFFPAgAAAAAMAAQAAAAAAAAAAAAAAAAAAFhkTvmBZkLp01e%2FXlJJKcn%2F%2F%2F%2F%2FAAAAAQAAAAAAAAAAAAAAAQAAAAzht9umOLxb19t+0NJhSuU7jsXlj51Q3o4XPw8F; zguid=24|%242ee18300-f1c2-4c4c-894d-e95cdb5a8765; zgsession=1|34c3097e-3313-40f4-bc8b-dda201d247e0; zjs_anonymous_id=%222ee18300-f1c2-4c4c-894d-e95cdb5a8765%22; zjs_user_id=null; zg_anonymous_id=%22fba31dc9-7da4-465a-87f0-e352d42726c6%22; _gcl_au=1.1.1962137471.1691074681; DoubleClickSession=true; pxcts=47bbb743-39c1-11ee-a406-4c704d515a67; _pxvid=47bba73a-39c1-11ee-a406-69b061c9f9e6; JSESSIONID=81E99BD7B574C62C2B6996CAFF097408; x-amz-continuous-deployment-state=AYABeOobNKWMlqXk6Opv4KqHZ00APgACAAFEAB1kM2Jsa2Q0azB3azlvai5jbG91ZGZyb250Lm5ldAABRwAVRzA3MjU1NjcyMVRZRFY4RDcyVlpWAAEAAkNEABpDb29raWUAAACAAAAADIPw8Tz1c%2FJkcGJ0sQAw+rocAg2V+6AzJbuvD%2Fa4s+Ylx4br+E+wY2L6P6cJzA68%2Fqwb7RVmdfTZaPwT20EtAgAAAAAMAAQAAAAAAAAAAAAAAAAAANqit%2FMngxR65bjOjBYJVm%2F%2F%2F%2F%2F%2FAAAAAQAAAAAAAAAAAAAAAQAAAAx+hQx+vBn9U086grypTwjpTYN7VgMM3GdrUppTTwjpTYN7VgMM3GdrUppTTwjpTYN7VgMM3GdrUppTTwjpTYN7VgMM3GdrUppTTwjpTYN7VgMM3GdrUppT; search=6|1695876639229%7Crect%3D32.80386902766574%252C-96.68895721435547%252C32.70728894920813%252C-96.85323715209961%26rid%3D38128%26disp%3Dmap%26mdm%3Dauto%26sort%3Dpriorityscore%26listPriceActive%3D1%26fs%3D1%26fr%3D0%26mmm%3D0%26rs%3D0%26ah%3D0%26singlestory%3D0%26abo%3D0%26garage%3D0%26pool%3D0%26ac%3D0%26waterfront%3D0%26finished%3D0%26unfinished%3D0%26cityview%3D0%26mountainview%3D0%26parkview%3D0%26waterview%3D0%26hoadata%3D1%263dhome%3D0%26commuteMode%3Ddriving%26commuteTimeOfDay%3Dnow%09%0938128%09%7B%22isList%22%3Atrue%2C%22isMap%22%3Atrue%7D%09%09%09%09%09; AWSALB=UT3+qcYR3tHuczo/TbzN8YUy1B0/uBnd1W1eCwJmlP4fdIlrjUnSL0VikXGyr5hCjVRJPoZVqpN1joR4OVqQxhClYX9Jfm9NllqrJ5hEwtA5Zm73ARi0S8rV4i+E; AWSALBCORS=UT3+qcYR3tHuczo/TbzN8YUy1B0/uBnd1W1eCwJmlP4fdIlrjUnSL0VikXGyr5hCjVRJPoZVqpN1joR4OVqQxhClYX9Jfm9NllqrJ5hEwtA5Zm73ARi0S8rV4i+E; x-amz-continuous-deployment-state=AYABeIs8bDpDqQvTOLxmullpMEoAPgACAAFEAB1kM2Jsa2Q0azB3azlvai5jbG91ZGZyb250Lm5ldAABRwAVRzA3MjU1NjcyMVRZRFY4RDcyVlpWAAEAAkNEABpDb29raWUAAACAAAAADLqrfBR3ecBLDhYnbgAwZtZr8viUp3066btA1CybuVxFoUj3kYCq9JKc46gmzh47FTKYM4PIfZ5Vep1OqhhkAgAAAAAMAAQAAAAAAAAAAAAAAAAAANLcxYezdf26FjHJLrj9XDH%2F%2F%2F%2F%2FAAAAAQAAAAAAAAAAAAAAAQAAAAyIYroULD24T6PCx5cudKnJavjLoGK4W1ukGYks; AWSALB=CmB/8DLiDb1ycFVjeKynVX6F8ahPkMrM+ZR3RoL8wCsSAh1Xp7ELPH/319qgZl8Y6wzpbYkwg3cRX0ab6OvPb1Lew9o179ZvkT51W3UGGIqDdVAIJFOMMVCD3b9r; AWSALBCORS=CmB/8DLiDb1ycFVjeKynVX6F8ahPkMrM+ZR3RoL8wCsSAh1Xp7ELPH/319qgZl8Y6wzpbYkwg3cRX0ab6OvPb1Lew9o179ZvkT51W3UGGIqDdVAIJFOMMVCD3b9r',
        'referer': f'https://www.zillow.com/homedetails/{zpid}_zpid/',
        'sec-ch-ua': '"Not.A/Brand";v="8", "Chromium";v="114", "Opera GX";v="100"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"Windows"',
        'sec-fetch-dest': 'empty',
        'sec-fetch-mode': 'cors',
        'sec-fetch-site': 'same-origin',
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36 OPR/100.0.0.0',
        }

    response = requests.request("GET", url, headers=headers, data=payload)

    json=response.json()
    text=json["data"]["property"]
    listingInfo=json["data"]["property"]["attributionInfo"]
    
    data_dict = {
    "Agent Name": listingInfo["agentName"],
    "Agent Phone Number": listingInfo["agentPhoneNumber"],
    "Agent License Number": listingInfo["agentLicenseNumber"],
    "Broker Name":listingInfo["brokerName"],
    "Broker Phone Number": listingInfo["brokerPhoneNumber"],
    "Time on Zillow": text["timeOnZillow"],
    "Page View Count": text["pageViewCount"],
    "Favorite Count": text["favoriteCount"],
    "Days on Zillow": text["daysOnZillow"]
    }
    pprint.pprint(data_dict)
    return

getInfo_fromDetailedPage(zpid)

7 days 1022 53 7
{'Agent License Number': None,
 'Agent Name': 'Brian C. Stiles, Broker',
 'Agent Phone Number': '615-477-3089',
 'Broker Name': 'PARKS',
 'Broker Phone Number': '615-826-4040',
 'Days on Zillow': 7,
 'Favorite Count': 53,
 'Page View Count': 1022,
 'Time on Zillow': '7 days'}


In [17]:
print(len(bigData))
def test2(dictionaryList, fileName):
    try:
        df_existing = pd.read_excel(fileName)
    except:
        print("File does not exist creating file: "+fileName)
        df_existing=None
        
    df_new=pd.DataFrame(dictionaryList)
    
    df_combined = pd.concat([df_existing, df_new], ignore_index=True)
    
    with pd.ExcelWriter(fileName, engine='openpyxl') as writer:
        df_combined.to_excel(writer, index=False, sheet_name='Sheet')
    print("Data is added to", fileName)
    adjustColumnWidth(fileName)
fileName="zillowtest.xlsx"
test2(bigData,fileName)


import pandas as pd

def store_toXlsx(dictionaryList,fileName):
    try:
        df_existing = pd.read_excel(fileName)
        print(fileName+" exists")
    except:
        print("File does not exist creating file: "+fileName)
        df_existing=None
    df_new = pd.DataFrame(dictionaryList)
    df_combined = pd.concat([df_existing, df_new], ignore_index=True)
    with pd.ExcelWriter(fileName, engine='openpyxl') as writer:
        df_combined.to_excel(writer, index=False, sheet_name='Sheet')
    print("Data is added to", fileName)
    




41
File does not exist creating file: zillowtest.xlsx
Data is added to zillowtest.xlsx
